In [2]:
#!pip install --quiet glvis
import numpy as np
import matplotlib.pyplot as plt
from  bfp import *
import mfem.ser as mfem
#from mfem.common.arg_parser import ArgParser
from glvis import glvis, GlvisData
import seaborn as sns
import pandas as pd
import os

In [3]:
class EDependentCoefficient(mfem.PyCoefficient):
    def __init__(self, xs_t_data, E_start, E_end):
        super(EDependentCoefficient, self).__init__()
        self.E_start = E_start
        self.E_end = E_end
        if isinstance(xs_t_data, (int, float)):
            self.constant = True
            self.constant_value = float(xs_t_data)
        else:
            self.constant = False
            self.xs_t_data = xs_t_data
            self.n_groups = len(xs_t_data)
            self.E_bins = np.linspace(E_start, E_end, self.n_groups + 1)

    def EvalValue(self, x):
        if self.constant:
            return self.constant_value
        E = x[1]
        for i in range(self.n_groups - 1):
            if np.isclose(E, self.E_bins[i+1]):
                return float(self.xs_t_data[i+1])
            if E <= self.E_bins[i] and E > self.E_bins[i+1]:
                return float(self.xs_t_data[i])
        return float(self.xs_t_data[-1])

class XDependentCoefficient(mfem.PyCoefficient):
    def __init__(self, xs_t_data, x_start, x_end):
        super(XDependentCoefficient, self).__init__()
        self.x_start = x_start
        self.x_end = x_end
        if isinstance(xs_t_data, (int, float)):
            self.constant = True
            self.constant_value = float(xs_t_data)
        else:
            self.constant = False
            self.xs_t_data = xs_t_data
            self.n_groups = len(xs_t_data)
            self.x_bins = np.linspace(x_start, x_end, self.n_groups + 1)

    def EvalValue(self, x):
        if self.constant:
            return self.constant_value

        x_val = x[0]
        for i in range(self.n_groups - 1):
            if np.isclose(x_val, self.x_bins[i+1]):
                return float(self.xs_t_data[i+1])
            if self.x_bins[i] <= x_val < self.x_bins[i+1]:
                return float(self.xs_t_data[i])
        return float(self.xs_t_data[-1])

class MyVectorCoefficient(mfem.VectorConstantCoefficient):
    def __init__(self, mu_coeff, S_coeff):
        super(MyVectorCoefficient, self).__init__(2)
        self.mu_coeff = mu_coeff 
        self.S_coeff = S_coeff

    def EvalValue(self, x):
        mu_val = self.mu_coeff.EvalValue(x) 
        S_val = self.S_coeff.EvalValue(x) 
        return [mu_val, S_val]
    
class InflowCoefficientSN(mfem.PyCoefficient):
    def __init__(self, inflow, mu):
        super(InflowCoefficientSN, self).__init__()
        self.inflow = inflow
        self.mu = mu
    def EvalValue(self, x):
        if self.mu > 0:
            return self.inflow
        else:
            return 0
        
class ConstantVector(mfem.VectorPyCoefficient):
    def __init__(self, constant):
        super(ConstantVector, self).__init__(1)
        self.constant = constant 
    def EvalValue(self, x):
        return self.constant

In [4]:
def test_EDependentCoefficient():
    xs_t_data = [0,1,2,3,4]
    coeff = EDependentCoefficient(xs_t_data, 1, 0.01)
    print(coeff.EvalValue([0.0, 1]))
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([0.0, 0.604]))
    print(coeff.EvalValue([0.0, 0.406]))
    print(coeff.EvalValue([0.0, 0.208]))
    print(coeff.EvalValue([0.0, 0.01]))

def test_EDependentCoefficient_Constant():
    coeff = EDependentCoefficient(40, 0, 1)
    print(coeff.EvalValue([0.0, 1]))
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([1.0, 0.604]))
    print(coeff.EvalValue([0.3, 0.406]))
    print(coeff.EvalValue([0.6, 0.208]))
    print(coeff.EvalValue([0.8, 0.01]))

#print("Test for E dependent Coefficient")
#test_EDependentCoefficient()
#print("Test for E dependent Constant Coefficient")
#test_EDependentCoefficient_Constant()

def test_XDependentCoefficientt():
    xs_t_data = [0,1,2,3,4]
    coeff = XDependentCoefficient(xs_t_data, 0, 1)
    print(coeff.EvalValue([0.0, 0.00]))
    print(coeff.EvalValue([0.2, 0.00]))
    print(coeff.EvalValue([0.4, 0.00]))
    print(coeff.EvalValue([0.6, 0.00]))
    print(coeff.EvalValue([0.8, 0.00]))
    print(coeff.EvalValue([1.0, 0.00]))

def test_XDependentCoefficient_Constant():
    coeff = XDependentCoefficient(80, 0.0, 1.0)
    print(coeff.EvalValue([0.0, 0.00]))
    print(coeff.EvalValue([0.2, 0.00]))
    print(coeff.EvalValue([0.4, 0.00]))
    print(coeff.EvalValue([0.6, 0.00]))
    print(coeff.EvalValue([0.8, 0.00]))
    print(coeff.EvalValue([1.0, 0.00]))

#print("Test for X dependent Coefficient")
#test_XDependentCoefficientt()
#print("Test for X dependent Constant Coefficient")
#test_XDependentCoefficient_Constant()

def test_MyVectorCoefficient():
    mu_data = [5]
    mu = XDependentCoefficient(mu_data, 0.0, 1.0)
    S_data = [0,1,2,3,4]
    S = EDependentCoefficient(S_data, 1.0, 0.01)
    coeff = MyVectorCoefficient(mu, S)
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([0.2, 0.604]))
    print(coeff.EvalValue([0.4, 0.406]))
    print(coeff.EvalValue([0.6, 0.208]))
    print(coeff.EvalValue([0.8, 0.01]))

print("Test for Vector Coefficient")
test_MyVectorCoefficient()


Test for Vector Coefficient
[5.0, 1.0]
[5.0, 2.0]
[5.0, 3.0]
[5.0, 4.0]
[5.0, 4.0]


In [5]:
nx = 10
nE = 10
x_start = 0.0
x_end = 1.0
E_start = 0.0
E_end = 1.0
N_ang = 4
order = 1
inflow = 10.0

#mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
mesh = mfem.Mesh(nx, nE, "QUADRILATERAL", True, x_end, E_end)
#for x in range(ref_levels):
mesh.UniformRefinement()
#mesh.UniformRefinement()
#mesh.UniformRefinement()
#mesh.UniformRefinement()
dim = mesh.Dimension()

# btype = 0-GaussLegendre, 1 - GaussLobatto
fec = mfem.DG_FECollection(order, dim, btype=0)
fes = mfem.FiniteElementSpace(mesh, fec)
Size = fes.GetVSize()
print("Number of unknowns:", Size)
print('Number of finite element unknowns: ' + str(fes.GetVSize()))

mu_vals, w_vals = gauss_legendre_dirs(N_ang)

#ex17'den


ess_tdof_list = mfem.intArray()
dir_bdr = mfem.intArray(mesh.bdr_attributes.Max())
dir_bdr.Assign(0)
dir_bdr[3] = 1
#dir_bdr[1] = 1

S_const = 0.0
xs_t_const = 5.0
#xs_s_const = 1.0
S_arr = np.ones(nE) * S_const
E_arr = np.linspace(E_start, E_end, nE+1)

S_coeff = EDependentCoefficient(S_const, E_start, E_end)
xs_t_coeff = EDependentCoefficient(xs_t_const, E_start, E_end)
#xs_s_coeff = EDependentCoefficient(xs_s_const, E_start, E_end)
q_coeff = EDependentCoefficient(0, E_start, E_end)

# Solution

alpha = 1.0
beta = 0.5

psi_mu = []
psi_mu_list = []

for mu, w in zip(mu_vals, w_vals):
    print("  Solving for mu =", mu)
    mu_coeff = ConstantCoefficient(mu)
    inflow_coeff = InflowCoefficientSN(inflow, mu)
    #marker = get_marker_for_mu(mesh, mu)
    #v_coeff = MyVectorCoefficient(mu_coeff, S_coeff)
    v_coeff = mfem.VectorConstantCoefficient([mu, 0.0])
    print(f"inflow: {inflow_coeff.EvalValue([inflow, mu])}")
    #print(f"v_coeff: {v_coeff.EvalValue([mu, 0.8])}")
    print(f"sigma_t: {xs_t_coeff.EvalValue([0.0, 0.604])}")


    a = mfem.BilinearForm(fes)
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coeff, 1.0))
    a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
    a.AddInteriorFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, -alpha, beta)))
    a.AddBdrFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, -alpha, beta)), dir_bdr)
    a.Assemble()
    a.Finalize()
    A = a.SpMat()

    b = mfem.LinearForm(fes)
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coeff))
    b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff, v_coeff, -alpha, -beta), dir_bdr)
    b.Assemble()
    
    psi = mfem.GridFunction(fes)
    psi.Assign(1.0)

    prec = mfem.GSSmoother(A)
    solver = mfem.GMRESSolver()
    solver.SetOperator(A)
    solver.SetPreconditioner(prec)
    solver.SetRelTol(1e-12)
    solver.SetAbsTol(1e-12)
    solver.SetMaxIter(10000)
    solver.SetKDim(30)
    solver.SetPrintLevel(1)
    solver.Mult(b, psi)
    

    g = glvis((mesh, psi), 500,500, keys='ARjmcbp')
    g.set_size(1000, 1000)
    g.render()
    
    #X_arr = mfem.GridFunction(fes)
    #X_arr.Assign(X)
    psi.Save("psi_hs_mu_{:.3f}.gf".format(mu))
    psi_mu.append(psi.GetDataArray())
    psi_mu_list.append((mu, w, psi))


phi = mfem.GridFunction(fes)
phi.Assign(0.0)
for mu, w, X in psi_mu_list:
    phi.Add(w, X)

phi.Save("phi_dg.gf")

    

Number of unknowns: 1600
Number of finite element unknowns: 1600
  Solving for mu = -0.8611363115940526
inflow: 0
sigma_t: 5.0
   Pass :  1   Iteration :   0  ||B r|| = 113.524
   Pass :  1   Iteration :   1  ||B r|| = 30.7429
   Pass :  1   Iteration :   2  ||B r|| = 19.8669
   Pass :  1   Iteration :   3  ||B r|| = 15.5451
   Pass :  1   Iteration :   4  ||B r|| = 12.6885
   Pass :  1   Iteration :   5  ||B r|| = 9.50996
   Pass :  1   Iteration :   6  ||B r|| = 7.3509
   Pass :  1   Iteration :   7  ||B r|| = 5.33324
   Pass :  1   Iteration :   8  ||B r|| = 3.60997
   Pass :  1   Iteration :   9  ||B r|| = 2.707
   Pass :  1   Iteration :  10  ||B r|| = 2.20154
   Pass :  1   Iteration :  11  ||B r|| = 2.00984
   Pass :  1   Iteration :  12  ||B r|| = 1.72407
   Pass :  1   Iteration :  13  ||B r|| = 1.41349
   Pass :  1   Iteration :  14  ||B r|| = 1.04805
   Pass :  1   Iteration :  15  ||B r|| = 0.805194
   Pass :  1   Iteration :  16  ||B r|| = 0.688067
   Pass :  1   Iteration

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

  Solving for mu = -0.33998104358485626
inflow: 0
sigma_t: 5.0
   Pass :  1   Iteration :   0  ||B r|| = 41.3181
   Pass :  1   Iteration :   1  ||B r|| = 16.8706
   Pass :  1   Iteration :   2  ||B r|| = 11.7547
   Pass :  1   Iteration :   3  ||B r|| = 7.05649
   Pass :  1   Iteration :   4  ||B r|| = 5.18946
   Pass :  1   Iteration :   5  ||B r|| = 4.45598
   Pass :  1   Iteration :   6  ||B r|| = 3.96954
   Pass :  1   Iteration :   7  ||B r|| = 3.90512
   Pass :  1   Iteration :   8  ||B r|| = 3.9007
   Pass :  1   Iteration :   9  ||B r|| = 3.74358
   Pass :  1   Iteration :  10  ||B r|| = 3.36877
   Pass :  1   Iteration :  11  ||B r|| = 2.79048
   Pass :  1   Iteration :  12  ||B r|| = 2.19641
   Pass :  1   Iteration :  13  ||B r|| = 1.37361
   Pass :  1   Iteration :  14  ||B r|| = 0.760587
   Pass :  1   Iteration :  15  ||B r|| = 0.385436
   Pass :  1   Iteration :  16  ||B r|| = 0.231765
   Pass :  1   Iteration :  17  ||B r|| = 0.136825
   Pass :  1   Iteration :  18  ||

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

  Solving for mu = 0.33998104358485626
inflow: 10.0
sigma_t: 5.0
   Pass :  1   Iteration :   0  ||B r|| = 83.329
   Pass :  1   Iteration :   1  ||B r|| = 56.943
   Pass :  1   Iteration :   2  ||B r|| = 37.5365
   Pass :  1   Iteration :   3  ||B r|| = 24.3728
   Pass :  1   Iteration :   4  ||B r|| = 17.5333
   Pass :  1   Iteration :   5  ||B r|| = 12.7086
   Pass :  1   Iteration :   6  ||B r|| = 8.86495
   Pass :  1   Iteration :   7  ||B r|| = 6.22169
   Pass :  1   Iteration :   8  ||B r|| = 4.33631
   Pass :  1   Iteration :   9  ||B r|| = 3.10848
   Pass :  1   Iteration :  10  ||B r|| = 2.09656
   Pass :  1   Iteration :  11  ||B r|| = 1.43154
   Pass :  1   Iteration :  12  ||B r|| = 0.92281
   Pass :  1   Iteration :  13  ||B r|| = 0.626784
   Pass :  1   Iteration :  14  ||B r|| = 0.371577
   Pass :  1   Iteration :  15  ||B r|| = 0.201701
   Pass :  1   Iteration :  16  ||B r|| = 0.0746589
   Pass :  1   Iteration :  17  ||B r|| = 0.0385819
   Pass :  1   Iteration :  18

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

  Solving for mu = 0.8611363115940526
inflow: 10.0
sigma_t: 5.0
   Pass :  1   Iteration :   0  ||B r|| = 291.647
   Pass :  1   Iteration :   1  ||B r|| = 142.954
   Pass :  1   Iteration :   2  ||B r|| = 105.968
   Pass :  1   Iteration :   3  ||B r|| = 86.3053
   Pass :  1   Iteration :   4  ||B r|| = 67.4148
   Pass :  1   Iteration :   5  ||B r|| = 55.523
   Pass :  1   Iteration :   6  ||B r|| = 43.7892
   Pass :  1   Iteration :   7  ||B r|| = 33.6283
   Pass :  1   Iteration :   8  ||B r|| = 25.5396
   Pass :  1   Iteration :   9  ||B r|| = 19.5771
   Pass :  1   Iteration :  10  ||B r|| = 14.8785
   Pass :  1   Iteration :  11  ||B r|| = 11.3868
   Pass :  1   Iteration :  12  ||B r|| = 8.33905
   Pass :  1   Iteration :  13  ||B r|| = 6.40734
   Pass :  1   Iteration :  14  ||B r|| = 4.97869
   Pass :  1   Iteration :  15  ||B r|| = 3.95889
   Pass :  1   Iteration :  16  ||B r|| = 2.70914
   Pass :  1   Iteration :  17  ||B r|| = 1.83603
   Pass :  1   Iteration :  18  ||B r

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

In [6]:
g = glvis((mesh, phi), 500,500, keys='ARjmcbp')
g.set_size(900, 900)
g.render()

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

In [7]:
g = glvis((mesh, psi), 500,500, keys='rljgac//0')
g.set_size(900, 900)
g.render()


_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…